<a href="https://colab.research.google.com/github/CptK1ng/dmc2019/blob/alexander_dev/notebooks/alexander_classificator_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificator v1 by Alexander


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, linear_model, metrics
%matplotlib inline

Download our custom Dataset splits:

In [0]:
!wget -nc -q --show-progress https://www.dropbox.com/s/6m8iq9ogpzmu7vx/train_new.csv?dl=1 -O train_new.csv
!wget -nc -q --show-progress https://www.dropbox.com/s/tjpkc45oqn3uv8s/val_new.csv?dl=1 -O val_new.csv

Import Data:

In [50]:
df_train_original = pd.read_csv("train_new.csv", sep="|")
df_val_original = pd.read_csv("val_new.csv", sep="|")
df_train_original.head(2)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,4,828,66.56,7,4,3,0.007246,0.080386,1.166667,0
1,1,1612,31.34,2,4,3,0.008685,0.019442,0.142857,0


## Feature Engineering

In [59]:
def prepareData(df):
  
  df = df.copy()
  """ this custom feature engineering does not improve the score...
  df['trustLevel'] = df['trustLevel']/6 #normalization
  df['totalLineItems'] = df['scannedLineItemsPerSecond'] * df['totalScanTimeInSeconds'] #nur of scanned products
  # df['totalValue'] = df['valuePerSecond'] * df['totalScanTimeInSeconds'] #this is equal to the column "grandTotal"
  # TODO df['lineItemVoids'] = df['lineItemVoidsPerPosition'] * df['totalLineItemVoidsPositions']
  df['lineItemVoidPercentage'] = df['lineItemVoids'] / (df['totalLineItems'] + df['scansWithoutRegistration'])
  df['totalPositions'] = df['totalLineItems'] - df['lineItemVoids'] # + df['quantityModifications']
  """
  return df

df_train = prepareData(df_train_original)
df_val = prepareData(df_val_original)

#df_train = df_train_original.copy()
#df_val = df_val_original.copy()

df_train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,4,828,66.56,7,4,3,0.007246,0.080386,1.166667,0
1,1,1612,31.34,2,4,3,0.008685,0.019442,0.142857,0
2,3,848,52.37,2,4,0,0.022406,0.061757,0.105263,0
3,1,321,76.03,8,7,2,0.071651,0.236854,0.347826,0
4,1,660,6.06,3,7,1,0.027273,0.009182,0.166667,0


## Split X and Y

In [0]:
# Splitting the final dataset into internal training and testing datasets
df_train_X = df_train.drop('fraud', axis=1)
df_train_y = df_train['fraud']
df_val_X = df_val.drop('fraud', axis=1)
df_val_y = df_val['fraud']
X_train, X_test, y_train, y_test = df_train_X.values, df_val_X.values, df_train_y.values, df_val_y.values

## Simple Classificator


In [64]:
# Fitting the model and printing the accuracy score
model = linear_model.SGDClassifier(loss='log', max_iter=4000, tol=1e-3, random_state=0) # Random state is fixed for reproducibility
#TODO: Implement our custom loss function
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.9388297872340425

In [65]:
# Printing the confusion matrix
metrics.confusion_matrix(y_test, y_pred)

array([[353,   0],
       [ 23,   0]])

## ADA Boost